In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Creating the datasets
house_data_train = pd.read_csv("train.csv")
house_data_test = pd.read_csv("test.csv")
# Target prices:
y = house_data_train['SalePrice']
house_data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [3]:
# Columns I will work with
columns = [
    'MSSubClass',
    'LotArea',
    'LotShape',
    'LandContour',
    'Utilities',
    'LandSlope',
    'Condition1',
    'Condition2',
    'BldgType',
    'HouseStyle',
    'OverallQual',
    'OverallCond',
    'Foundation',
    '1stFlrSF',
    '2ndFlrSF',
    'BedroomAbvGr',
    'KitchenAbvGr',
    'FullBath',
    'HalfBath',
    'GarageArea',
    'PoolArea'
]

In [4]:
# Partial datasets
partial_train = house_data_train[columns]
partial_test = house_data_test[columns]
partial_train 

,MSSubClass,LotArea,LotShape,LandContour,Utilities,LandSlope,Condition1,Condition2,BldgType,HouseStyle,...,OverallCond,Foundation,1stFlrSF,2ndFlrSF,BedroomAbvGr,KitchenAbvGr,FullBath,HalfBath,GarageArea,PoolArea
0,60,8450,Reg,Lvl,AllPub,Gtl,Norm,Norm,1Fam,2Story,...,5,PConc,856,854,3,1,2,1,548,0
1,20,9600,Reg,Lvl,AllPub,Gtl,Feedr,Norm,1Fam,1Story,...,8,CBlock,1262,0,3,1,2,0,460,0
2,60,11250,IR1,Lvl,AllPub,Gtl,Norm,Norm,1Fam,2Story,...,5,PConc,920,866,3,1,2,1,608,0
3,70,9550,IR1,Lvl,AllPub,Gtl,Norm,Norm,1Fam,2Story,...,5,BrkTil,961,756,3,1,1,0,642,0
4,60,14260,IR1,Lvl,AllPub,Gtl,Norm,Norm,1Fam,2Story,...,5,PConc,1145,1053,4,1,2,1,836,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,7917,Reg,Lvl,AllPub,Gtl,Norm,Norm,1Fam,2Story,...,5,PConc,953,694,3,1,2,1,460,0
1456,20,13175,Reg,Lvl,AllPub,Gtl,Norm,Norm,1Fam,1Story,...,6,CBlock,2073,0,3,1,2,0,500,0
1457,70,9042,Reg,Lvl,AllPub,Gtl,Norm,Norm,1Fam,2Story,...,9,Stone,1188,1152,4,1,2,0,252,0
1458,20,9717,Reg,Lvl,AllPub,Gtl,Norm,Norm,1Fam,1Story,...,6,CBlock,1078,0,2,1,1,0,240,0


In [5]:
# Values and meanings from the first column, MSSubClass
MSSubClass_dict = { 
        20:	'1-STORY 1946 & NEWER ALL STYLES', 
        30:	'1-STORY 1945 & OLDER', 
        40:	'1-STORY W/FINISHED ATTIC ALL AGES',
        45:	'1-1/2 STORY - UNFINISHED ALL AGES', 
        50:	'1-1/2 STORY FINISHED ALL AGES', 
        60:	'2-STORY 1946 & NEWER', 
        70:	'2-STORY 1945 & OLDER', 
        75:	'2-1/2 STORY ALL AGES', 
        80:	'SPLIT OR MULTI-LEVEL', 
        85:	'SPLIT FOYER', 
        90:	'DUPLEX - ALL STYLES AND AGES', 
       120:	'1-STORY PUD (Planned Unit Development) - 1946 & NEWER', 
       150:	'1-1/2 STORY PUD - ALL AGES', 
       160:	'2-STORY PUD - 1946 & NEWER', 
       180:	'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER', 
       190:	'2 FAMILY CONVERSION - ALL STYLES AND AGES'}

# Replacing numbers with column names
house_data_train['MSSubClass'].replace(MSSubClass_dict, inplace=True)
house_data_test['MSSubClass'].replace(MSSubClass_dict, inplace=True)
MSSubClass_df_train = pd.get_dummies(house_data_train['MSSubClass'])
MSSubClass_df_test = pd.get_dummies(house_data_test['MSSubClass'])
MSSubClass_df_train

,1-1/2 STORY - UNFINISHED ALL AGES,1-1/2 STORY FINISHED ALL AGES,1-STORY 1945 & OLDER,1-STORY 1946 & NEWER ALL STYLES,1-STORY PUD (Planned Unit Development) - 1946 & NEWER,1-STORY W/FINISHED ATTIC ALL AGES,2 FAMILY CONVERSION - ALL STYLES AND AGES,2-1/2 STORY ALL AGES,2-STORY 1945 & OLDER,2-STORY 1946 & NEWER,2-STORY PUD - 1946 & NEWER,DUPLEX - ALL STYLES AND AGES,PUD - MULTILEVEL - INCL SPLIT LEV/FOYER,SPLIT FOYER,SPLIT OR MULTI-LEVEL
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1456,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1457,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1458,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [6]:
# One hot encoder for the column MSSubClass
MSSubClass_df_train = pd.get_dummies(house_data_train['MSSubClass'])
MSSubClass_df_test = pd.get_dummies(house_data_test['MSSubClass'])
MSSubClass_df_train

,1-1/2 STORY - UNFINISHED ALL AGES,1-1/2 STORY FINISHED ALL AGES,1-STORY 1945 & OLDER,1-STORY 1946 & NEWER ALL STYLES,1-STORY PUD (Planned Unit Development) - 1946 & NEWER,1-STORY W/FINISHED ATTIC ALL AGES,2 FAMILY CONVERSION - ALL STYLES AND AGES,2-1/2 STORY ALL AGES,2-STORY 1945 & OLDER,2-STORY 1946 & NEWER,2-STORY PUD - 1946 & NEWER,DUPLEX - ALL STYLES AND AGES,PUD - MULTILEVEL - INCL SPLIT LEV/FOYER,SPLIT FOYER,SPLIT OR MULTI-LEVEL
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1456,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1457,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1458,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [7]:
# Dropping the categorical column, MSSubClass
partial_train.drop(columns=['MSSubClass'])
partial_test.drop(columns=['MSSubClass'])
# Appending the One Hot Encoder to the partial datasets
partial_train = pd.concat([MSSubClass_df_train, partial_train], axis=1)
partial_test = pd.concat([MSSubClass_df_test, partial_test], axis=1)
partial_train


,1-1/2 STORY - UNFINISHED ALL AGES,1-1/2 STORY FINISHED ALL AGES,1-STORY 1945 & OLDER,1-STORY 1946 & NEWER ALL STYLES,1-STORY PUD (Planned Unit Development) - 1946 & NEWER,1-STORY W/FINISHED ATTIC ALL AGES,2 FAMILY CONVERSION - ALL STYLES AND AGES,2-1/2 STORY ALL AGES,2-STORY 1945 & OLDER,2-STORY 1946 & NEWER,...,OverallCond,Foundation,1stFlrSF,2ndFlrSF,BedroomAbvGr,KitchenAbvGr,FullBath,HalfBath,GarageArea,PoolArea
0,0,0,0,0,0,0,0,0,0,1,...,5,PConc,856,854,3,1,2,1,548,0
1,0,0,0,1,0,0,0,0,0,0,...,8,CBlock,1262,0,3,1,2,0,460,0
2,0,0,0,0,0,0,0,0,0,1,...,5,PConc,920,866,3,1,2,1,608,0
3,0,0,0,0,0,0,0,0,1,0,...,5,BrkTil,961,756,3,1,1,0,642,0
4,0,0,0,0,0,0,0,0,0,1,...,5,PConc,1145,1053,4,1,2,1,836,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,0,0,0,0,0,0,1,...,5,PConc,953,694,3,1,2,1,460,0
1456,0,0,0,1,0,0,0,0,0,0,...,6,CBlock,2073,0,3,1,2,0,500,0
1457,0,0,0,0,0,0,0,0,1,0,...,9,Stone,1188,1152,4,1,2,0,252,0
1458,0,0,0,1,0,0,0,0,0,0,...,6,CBlock,1078,0,2,1,1,0,240,0


In [8]:
# Meanings of the Condition1 and Condition2 column values
cond = {
    'Artery':	'Adjacent to arterial street',
    'Feedr':	'Adjacent to feeder street',	
    'Norm':	'Normal',	
    'RRNn':	'Within 200 of North-South Railroad',
    'RRAn':	'Adjacent to North-South Railroad',
    'PosN':	'Near positive off-site feature--park, greenbelt, etc.',
    'PosA':	'Adjacent to postive off-site feature',
    'RRNe':	'Within 200 of East-West Railroad',
    'RRAe':	'Adjacent to East-West Railroad'
}

In [9]:
# Getting the full name of the conditions
partial_train['Condition1'].replace(cond, inplace=True)
partial_train['Condition2'].replace(cond, inplace=True)
# Getting the one hot encoder for every condition and dropping the column "Normal"
# because is it usually with a one (1)
c1_train = pd.get_dummies(partial_train["Condition1"]).rename(columns=cond).drop(columns=["Normal"])
c2_train = pd.get_dummies(partial_train["Condition2"]).rename(columns=cond).drop(columns=["Normal"])
# Sum both one hot encoders. Exploring this data, I found some 2's, that makes 
# no sense, so, I replaced them with 1's
c1c2_train = c1_train.add(c2_train, fill_value=0).replace(2,1)
c1c2_train

,Adjacent to East-West Railroad,Adjacent to North-South Railroad,Adjacent to arterial street,Adjacent to feeder street,Adjacent to postive off-site feature,"Near positive off-site feature--park, greenbelt, etc.",Within 200 of East-West Railroad,Within 200 of North-South Railroad
0,0,0,0,0,0,0,0.0,0
1,0,0,0,1,0,0,0.0,0
2,0,0,0,0,0,0,0.0,0
3,0,0,0,0,0,0,0.0,0
4,0,0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...
1455,0,0,0,0,0,0,0.0,0
1456,0,0,0,0,0,0,0.0,0
1457,0,0,0,0,0,0,0.0,0
1458,0,0,0,0,0,0,0.0,0


In [10]:
# Same process to the testing data
partial_test['Condition1'].replace(cond, inplace=True)
partial_test['Condition2'].replace(cond, inplace=True)
c1_test = pd.get_dummies(partial_test["Condition1"]).rename(columns=cond).drop(columns=["Normal"])
c2_test = pd.get_dummies(partial_test["Condition2"]).rename(columns=cond).drop(columns=["Normal"])
c1c2_test = c1_test.add(c2_test, fill_value=0).replace(2,1)

In [11]:
# Dropping categorical columns and appending the one hot encoders
partial_train = partial_train.drop(columns=["Utilities", "Condition1", "Condition2"])
partial_train = pd.concat([partial_train, c1c2_train], axis=1)
partial_test = partial_test.drop(columns=["Utilities", "Condition1", "Condition2"])
partial_test = pd.concat([partial_test, c1c2_test], axis=1)
partial_train

,1-1/2 STORY - UNFINISHED ALL AGES,1-1/2 STORY FINISHED ALL AGES,1-STORY 1945 & OLDER,1-STORY 1946 & NEWER ALL STYLES,1-STORY PUD (Planned Unit Development) - 1946 & NEWER,1-STORY W/FINISHED ATTIC ALL AGES,2 FAMILY CONVERSION - ALL STYLES AND AGES,2-1/2 STORY ALL AGES,2-STORY 1945 & OLDER,2-STORY 1946 & NEWER,...,GarageArea,PoolArea,Adjacent to East-West Railroad,Adjacent to North-South Railroad,Adjacent to arterial street,Adjacent to feeder street,Adjacent to postive off-site feature,"Near positive off-site feature--park, greenbelt, etc.",Within 200 of East-West Railroad,Within 200 of North-South Railroad
0,0,0,0,0,0,0,0,0,0,1,...,548,0,0,0,0,0,0,0,0.0,0
1,0,0,0,1,0,0,0,0,0,0,...,460,0,0,0,0,1,0,0,0.0,0
2,0,0,0,0,0,0,0,0,0,1,...,608,0,0,0,0,0,0,0,0.0,0
3,0,0,0,0,0,0,0,0,1,0,...,642,0,0,0,0,0,0,0,0.0,0
4,0,0,0,0,0,0,0,0,0,1,...,836,0,0,0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,0,0,0,0,0,0,1,...,460,0,0,0,0,0,0,0,0.0,0
1456,0,0,0,1,0,0,0,0,0,0,...,500,0,0,0,0,0,0,0,0.0,0
1457,0,0,0,0,0,0,0,0,1,0,...,252,0,0,0,0,0,0,0,0.0,0
1458,0,0,0,1,0,0,0,0,0,0,...,240,0,0,0,0,0,0,0,0.0,0


In [12]:
def dummies(df, column):
    '''This function accepts a dataframe and a column name. It will get the 
    one hot encoder for categorical column, then, drop it from the original
    dataframe and cancatenate the one hot encoder'''
    df1 = pd.get_dummies(df[column])
    df = df.drop(columns=[column])
    df = pd.concat([df1, df], axis=1)
    return df

In [13]:
# Using the custom function in the left categorical data columns.
partial_train= dummies(partial_train, 'BldgType')
partial_train= dummies(partial_train, 'HouseStyle')
partial_train= dummies(partial_train, 'Foundation')
partial_train= dummies(partial_train, 'LotShape')
partial_train= dummies(partial_train, 'LandContour')
partial_train= dummies(partial_train, 'LandSlope')

partial_test= dummies(partial_test, 'BldgType')
partial_test= dummies(partial_test, 'HouseStyle')
partial_test= dummies(partial_test, 'Foundation')
partial_test= dummies(partial_test, 'LotShape')
partial_test= dummies(partial_test, 'LandContour')
partial_test= dummies(partial_test, 'LandSlope')

In [14]:
from sklearn import preprocessing


x_train = partial_train.values #returns a numpy array
yv_train = y.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_train_scaled = min_max_scaler.fit_transform(x_train)
y_train_scaled = min_max_scaler.fit_transform(yv_train.reshape(-1, 1))

x_test = partial_test.values #returns a numpy array
x_test_scaled = min_max_scaler.fit_transform(x_test)
# Filling NaN values with 0
x_test_scaled[np.isnan(x_test_scaled)] = 0

In [15]:
from sklearn import svm

regr = svm.SVR()
regr.fit(x_train_scaled, yv_train)
predictions = regr.predict(x_test_scaled)

In [16]:
import csv
j = 1461
with open('predictions.csv', 'w') as p:
    writer = csv.writer(p)
    writer.writerow(['Id','SalePrice'])
    for pred in predictions:
        writer.writerow([j,pred])
        j += 1



In [17]:
from sklearn.linear_model import LinearRegression
LinearRegression = LinearRegression()
LinearRegression.fit(x_train_scaled, yv_train)
predictions2 = LinearRegression.predict(x_test_scaled)

In [18]:
import csv
j = 1461
with open('predictions2.csv', 'w') as p:
    writer = csv.writer(p)
    writer.writerow(['Id','SalePrice'])
    for pred in predictions2:
        writer.writerow([j,pred])
        j += 1